#### Calculate Sharpe, Sortino, Omega ratios for number of days in DAYS_RATIO
#### Save ratios to dest_path


In [25]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# # risk free rate
# ANNUAL_RISK_FREE_RATE = 0.04

In [26]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', 2500)        # Let the display adjust to the window


In [27]:
# process_files.py
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR, ANNUAL_RISK_FREE_RATE
from pathlib import Path  # Better path handling

print(f"DATE_STR: {DATE_STR}")
print(f"DEST_DIR: {DEST_DIR}")
print(f"ANNUAL_RISK_FREE_RATE: {ANNUAL_RISK_FREE_RATE}\n")

# Build paths
# source_path = Path(DEST_DIR) / f'{DATE_STR}_df_OHLCV_clean_stocks_etfs.parquet'
source_path = Path(DEST_DIR) / f'df_OHLCV_clean_stocks_etfs.parquet'
dest_path = Path(DEST_DIR) / f'{DATE_STR}_df_perf_ratios_stocks_etfs.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

DATE_STR: 2025-05-14
DEST_DIR: ..\data
ANNUAL_RISK_FREE_RATE: 0.04

source_path: ..\data\df_OHLCV_clean_stocks_etfs.parquet
dest_path: ..\data\2025-05-14_df_perf_ratios_stocks_etfs.parquet


In [28]:
import pandas as pd

# Get tickers from df_finviz_stocks_etfs.pkl file
df_finviz = pd.read_parquet(f'..\data\{DATE_STR}_df_finviz_stocks_etfs.parquet')
tickers = df_finviz.index.to_list()

In [29]:
print(df_finviz) 

        No.                                           Company               Index                  Sector                        Industry Country Exchange  Market Cap, M    P/E  Fwd P/E   PEG    P/S    P/B    P/C   P/FCF  Book/sh  Cash/sh  Dividend %  Dividend TTM Dividend Ex Date  Payout Ratio %    EPS  EPS next Q  EPS this Y %  EPS next Y %  EPS past 5Y %  EPS next 5Y %  Sales past 5Y %  Sales Q/Q %  EPS Q/Q %  EPS YoY TTM %  Sales YoY TTM %  Sales, M  Income, M  EPS Surprise %  Revenue Surprise %  Outstanding, M  Float, M  Float %  Insider Own %  Insider Trans %  Inst Own %  Inst Trans %  Short Float %  Short Ratio  Short Interest, M  ROA %   ROE %  ROI %  Curr R  Quick R  LTDebt/Eq  Debt/Eq  Gross M %  Oper M %  Profit M %  Perf Week %  Perf Month %  Perf Quart %  Perf Half %  Perf Year %  Perf YTD %  Beta    ATR  Volatility W %  Volatility M %  SMA20 %  SMA50 %  SMA200 %  50D High %  50D Low %  52W High %  52W Low %        52W Range  All-Time High %  All-Time Low %    RSI  Earning

In [30]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
A      2025-05-14   111.5200
       2025-05-13   115.4200
       2025-05-12   115.5500
       2025-05-09   106.9300
       2025-05-08   108.7000
...                      ...
ZWS    2024-02-07    32.7917
       2024-02-06    31.4780
       2024-02-05    30.8064
       2024-02-02    30.4805
       2024-02-01    29.8878

[503286 rows x 1 columns]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 503286 entries, ('A', Timestamp('2025-05-14 00:00:00')) to ('ZWS', Timestamp('2024-02-01 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  503286 non-null  float64
dtypes: float64(1)
memory usage: 5.8+ MB


In [32]:
import pandas as pd

def select_tickers_data(df, tickers, end_date=None):
    """
    Selects data for a list of tickers from a DataFrame with a MultiIndex
    where the first level is the ticker and the second level is the date.
    Optionally filters the data to include only dates up to and including end_date.

    Args:
        df (pd.DataFrame): The input DataFrame with a MultiIndex
                           where the first level is the ticker and the second
                           level is the date.
        tickers (list): A list of ticker symbols to select.
        end_date (str or pd.Timestamp, optional): The last date to include
                                                  in the returned DataFrame.
                                                  If None (default), all dates are included.
                                                  Can be a string that pandas can parse as a date
                                                  or a pandas Timestamp object.

    Returns:
        pd.DataFrame: A DataFrame containing only the data for the specified tickers
                      and dates up to the end_date (if provided).
                      Rows for tickers not found will not be included.
    """
    valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

    if not valid_tickers:
        print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
        return pd.DataFrame()

    try:
        ticker_df = df.loc[valid_tickers]
        if end_date is not None:
            if isinstance(end_date, str):
                end_date = pd.to_datetime(end_date)
            # Ensure the second level of the MultiIndex is datetime objects for comparison
            if not isinstance(ticker_df.index.get_level_values(1)[0], pd.Timestamp):
                ticker_df.index = ticker_df.index.set_levels(pd.to_datetime(ticker_df.index.get_level_values(1)), level=1)
            ticker_df = ticker_df[ticker_df.index.get_level_values(1) <= end_date]
        return ticker_df
    except KeyError as e:
        print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
        return pd.DataFrame()

In [33]:
selected_data = select_tickers_data(df, tickers, end_date=DATE_STR)
print(selected_data)

                   Adj Close
Symbol Date                 
MSFT   2025-05-14   452.9400
       2025-05-13   449.1400
       2025-05-12   449.2600
       2025-05-09   438.7300
       2025-05-08   438.1700
...                      ...
SPHD   2024-02-07    39.7676
       2024-02-06    39.8921
       2024-02-05    39.5666
       2024-02-02    40.1888
       2024-02-01    40.5813

[492016 rows x 1 columns]


In [34]:
selected_data.loc["A"]

,Adj Close
Date,
2025-05-14,111.520
2025-05-13,115.420
2025-05-12,115.550
2025-05-09,106.930
2025-05-08,108.700
...,...
2024-02-07,131.595
2024-02-06,134.638
2024-02-05,131.625


In [35]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [36]:
print(DAYS_RATIO)
print(DATE_STR)

[3, 5, 10, 15, 30, 60, 120, 250]
2025-05-14


In [37]:
print(len(list_dfs[0].loc["A"]))
print(list_dfs[0].loc["A"])


3
            Adj Close
Date                 
2025-05-12     115.55
2025-05-13     115.42
2025-05-14     111.52


In [38]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=ANNUAL_RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
      Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d   Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A    -12.107300  -11.641133  0.000000   2.255932    6.052580   1.556961    2.550838     5.518188   1.597274    1.806909     3.762395   1.433312   -0.443804    -0.640653   0.925867   -1.928596    -2.585386   0.706392    -0.661716     -0.928766    0.887514    -1.018126     -1.368835    0.830139
AA    39.844809         inf       NaN  16.809128         inf        NaN    9.721913    37.466395   5.255741    4.002854    10.146216   1.987329   -0.027722    -0.039850   0.995222   -0.901582    -1.235525   0.857737    -1.381585     -1.826163    0.783312    -0.371098     -0.519822    0.939489
AAL    1.998362    4.862360  1.433173   6.805212   15.556456   2.959929    9.

In [39]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1516 entries, A to ZWS
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sharpe 3d     1516 non-null   float64
 1   Sortino 3d    1516 non-null   float64
 2   Omega 3d      1141 non-null   float64
 3   Sharpe 5d     1516 non-null   float64
 4   Sortino 5d    1516 non-null   float64
 5   Omega 5d      1367 non-null   float64
 6   Sharpe 10d    1516 non-null   float64
 7   Sortino 10d   1516 non-null   float64
 8   Omega 10d     1506 non-null   float64
 9   Sharpe 15d    1516 non-null   float64
 10  Sortino 15d   1516 non-null   float64
 11  Omega 15d     1516 non-null   float64
 12  Sharpe 30d    1516 non-null   float64
 13  Sortino 30d   1516 non-null   float64
 14  Omega 30d     1516 non-null   float64
 15  Sharpe 60d    1516 non-null   float64
 16  Sortino 60d   1516 non-null   float64
 17  Omega 60d     1516 non-null   float64
 18  Sharpe 120d   1516 non-null   floa

In [40]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            # NaN is from Omega ratio where daily loss series are all zero
            # +Inf is from Sharpe ratio where daily negative excess return series are all zero
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            # -Inf is from Sharpe ratio where daily excess return series are the same, therefore standard deviation is zero
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['BSCQ', 'FTSM', 'RITM']
Column: Sortino 3d
  Inf: ['AA', 'ABEV', 'ACHR', 'ACM', 'ACWI', 'ADBE', 'ADI', 'AER', 'AL', 'ALSN', 'AM', 'AMAT', 'AMD', 'AMP', 'AMX', 'ANET', 'ANSS', 'APP', 'APTV', 'ARKK', 'ARM', 'AS', 'ASML', 'ASR', 'ASX', 'ATI', 'AVEM', 'AXON', 'BA', 'BAC', 'BAM', 'BAP', 'BBCA', 'BBUS', 'BBVA', 'BBWI', 'BCH', 'BCS', 'BEKE', 'BIRK', 'BLK', 'BMA', 'BMO', 'BN', 'BNS', 'BNT', 'BPOP', 'BR', 'BROS', 'BSAC', 'BSCP', 'BURL', 'BXSL', 'BYD', 'CART', 'CCJ', 'CCL', 'CDNS', 'CELH', 'CFLT', 'CG', 'CGGO', 'CGGR', 'CGXU', 'CHDN', 'CHRW', 'CHTR', 'CHWY', 'CLH', 'CLS', 'CM', 'CNI', 'COHR', 'COIN', 'COOP', 'COR', 'CP', 'CPNG', 'CPRT', 'CQP', 'CRDO', 'CRK', 'CRM', 'CRS', 'CSX', 'CTRA', 'CVNA', 'CW', 'CWB', 'CWEN', 'CWEN-A', 'CZR', 'DAR', 'DB', 'DDOG', 'DELL', 'DFAE', 'DFEM', 'DFUS', 'DINO', 'DIS', 'DRI', 'DSI', 'DT', 'DTM', 'DUOL', 'DYNF', '

In [41]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'AA'

#### Use Cell below to check the inf, -inf and NaN replacement

In [42]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       39.844809
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       16.809128
Sortino 5d            inf
Omega 5d              NaN
Sharpe 10d       9.721913
Sortino 10d     37.466395
Omega 10d        5.255741
Sharpe 15d       4.002854
Sortino 15d     10.146216
Omega 15d        1.987329
Sharpe 30d      -0.027722
Sortino 30d     -0.039850
Omega 30d        0.995222
Sharpe 60d      -0.901582
Sortino 60d     -1.235525
Omega 60d        0.857737
Sharpe 120d     -1.381585
Sortino 120d    -1.826163
Omega 120d       0.783312
Sharpe 250d     -0.371098
Sortino 250d    -0.519822
Omega 250d       0.939489
Name: AA, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d         39.844809
Sortino 3d      8267.251809
Omega 3d         737.505329
Sharpe 5d         16.809128
Sortino 5d      4877.541867
Omega 5d         615.512514
Sharpe 10d         9.721913
Sortino 10d       37.466395
Omega 10d          5.255741
Sharpe 15d         4.002854
Sortino 15d       10.146216
Omega 15d          1.987329
Sharpe 30d        -0.027722
Sortino 30d       -0.039850
Omega 30d          0.995222
Sharpe 60d        -0.901582
Sortino 60d       -1.235525
Omega 60d          0.857737
Sharpe 120d       -1.381585
Sortino 120d      -1.826163
Omega 120d         0.783312
Sharpe 250d       -0.371098
Sortino 250d      -0.519822
Omega 250d         0.939489
Name: AA, dtype: float64

In [43]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-05-14_df_perf_ratios_stocks_etfs.parquet


In [44]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'AMZN', 'TSLA', 'META', 'NFLX', 'V', 'JPM', 'GLD', 'MSTR', 'IBIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(f"DATE_STR: {DATE_STR}")
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

DATE_STR: 2025-05-14
+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============